# Exercise - Data

<div class="alert alert-block alert-success">
This is an exercise for understanding the relationships between entities and their attributes based on Chapter 4 “Data”.

## Content <a id="sec_toc"> </a> 

[Learning Objectives](#sec_0)

[a) Table-based Data Storage](#sec_a)

[b) Optimize Data Structure for Storage](#sec_b)

[c) Visualization of Data Structures with ERDs](#sec_c)

[d) Spit Table in Smaller Tables](#sec_d)

[e) Store Tables in a Database](#sec_e)

[f) Compare Memory Requirements](#sec_f)

### <a id="sec_0">Learning Objectives</a>

* Understand data structures, their optimization and visualization.
* Interpret and work with entity–relationship diagrams (ERDs).
* Describe how entities are represented in table based structures.
* Compare memory requirements of storing data in a csv table versus in a database.

Import the needed libaries:

In [ ]:
from enum import Enum

from typing import List

import pandas as pd 
import random as r
import sqlite3
import os
import numpy as np

## <a id="sec_a">a) Table-based Data Storage</a> 

The objective of this exercise is to strengthen your understanding of data structures and their optimization and visualization. To learn how to efficiently organize and store data, we will use an example. First, we start with a CSV file that contains all information about a university, including:

* departments 
* faculties 
* lectures
* students 
* majors

All information is stored in a single CSV table. 

Before optimizing the data storage, let's understand the data and identify potetial problems.

<u>TASK:</u> Inspect the table below:
* Which information does the table include? 
* What does one row represent? 

In [ ]:
# DO NOT CHANGE

df = pd.read_csv("university_big.csv")
df

The following code cell prints the dimension of the table. Each row represents one enrollment of a student in a lecture, which means that the information of the student, the lecture, the faculty member, and the department is repeated for every enrolled student. 

In [ ]:
# DO NOT CHANGE

df.shape

In the following, the frequency of identical values in the table is analyzed to identify redundancy.

In [ ]:
# DO NOT CHANGE

print(df["department_name"].value_counts(), "\n")
print(df["faculty_name"].value_counts(), "\n")
print(df["lecture_name"].value_counts())

<u>TASK:</u> Answer the following questions:
* Why are details about departments, faculty and lectures repeated many times in the table? 
* Does this repeated information represent new data, or is it the same information shown again?
* What problems would occur if information changes, for example a department name or a faculty-department relationship?

[Back](#sec_toc)

## <a id="sec_b">b) Optimize Data Structure for Storage</a>

To start omptimzing the data structure, real-world objects that exist independently have to be identified. In this example these are: 
* Student 
* Major 
* Lecture
* Faculty 
* Department 

The code cell below prints all column headers in the table.\
<u>TASK:</u> Group the columns into logical units:
* Student information
* Major information
* Lecture information
* Faculty information
* Department information

In [ ]:
# DO NOT CHANGE

df.columns

[Back](#sec_toc)

## <a id="sec_c"> c) Visualization of Data Structures with ERDs </a> 

To identify separate entities, define relationships and store information efficiently, we use an Entity–Relationship Diagram (ERD). It helps to design a data structure that avoids redundancy by splitting information into multiple tables and referencing them using unique identifiers, called primary keys.

Below you can see an example of an Entity-Relationship Diagram (ERD). The model represents the storage of information about movies. In the lecture, you were already introduced to entities, attributes, and their relationships. Within this example, we can identify the following components: 

- Entities: Movie, Person, Country
- Attributes: name, first_name, last_name, population, duration, date_of_birth, area, etc.
- Relationships: The lines connecting the entities represent one-to-one, one-to-many (Director → Movie) or many-to-many (Movie ↔ Country) relationships, depending on the symbols at the end of the line. 

```mermaid
erDiagram
    Movie {
        str name
        time duration
        date release_date
        int production_costs
    }
    Director {
        str first_name
        str last_name
        date date_of_birth
    }
    Country {
        str name 
        int population 
        float area
        Continent continent
    }

    Movie }o--|| Director : directed_by
    Movie }o--o{ Country : produced_in

As a next step, the university exmaple will be transformed into a ERD diagram to outline the structure of the new data structure.

```mermaid
erDiagram
    Student {
        str name
        int matriculation_number
    }
    Lecture {
        str name
        int lecture_number
    }
    Faculty {
        str name 
        int personnel_number 
        Role role
        bool department_head
    }
    Department {
        str name
        int dep_number
    }
    Major {
        str name
        int major_number
        Degree degree_type
    }



    Lecture }o--|| Faculty : held_by
    Lecture }o--|| Department : offered_by
    Lecture }o--o{ Student : attendees
    Faculty }o--|| Department : member_of
    Student }o--|| Major : studies

[Back](#sec_toc)

## <a id="sec_d"> d) Spit Table in Smaller Tables</a>

As the next step, the structure outlined in the ERD is put into practice by splitting the large table into smaller tables.

Each smaller table represents an entity from the ERD and contains all its attributes. One-to-many relationships are implemented directly within the relevant tables, while many-to-many relationships are represented using separate relationship tables that reference the ralted entities via their respective primary keys. 

In these tables, and also in the original large table, only lectures with enrolled students apear. Lectures without any enrolled students are therefore not included in the data. 

Furthermore, the following restrictions apply:
- Students: Only students who are enrolled in offered lectures are stored.
- Lectures: Only lectures that have at least one enrolled student are included.
- Faculty: Only faculty members teaching one of these lectures are listed.
- Majors: Only majors that are actively taken by the listed students are shown.

**Student Table**

In [ ]:
# DO NOT CHANGE

students = df[[
    "student_name",
    "matriculation_number",
    "major_number"
]].drop_duplicates().reset_index(drop=True)
students.head()

**Major Table**

In [ ]:
# DO NOT CHANGE

majors = df[[
    "major_name",
    "major_number",
    "degree_type"
]].drop_duplicates().reset_index(drop=True)
majors

**Department Table**

In [ ]:
# DO NOT CHANGE

departments = df[[
    "department_name",
    "dep_number"
]].drop_duplicates().reset_index(drop=True)
departments

**Faculty Table**

In [ ]:
# DO NOT CHANGE

faculty = df[[
    "faculty_name",
    "personnel_number",
    "role",
    "department_head",
    "dep_number"
]].drop_duplicates().reset_index(drop=True)
faculty

**Lecture Table**

In [ ]:
# DO NOT CHANGE

lectures = df[[
    "lecture_name",
    "lecture_number",
    "personnel_number",
    "dep_number"
]].drop_duplicates().reset_index(drop=True)
lectures

**Enrollement Table**\
This table shows the relationship between students and the lectures they are enrolled in.

In [ ]:
# DO NOT CHANGE

enrollments = df[[
    "lecture_number",
    "matriculation_number"
]].drop_duplicates().reset_index(drop=True)
enrollments.head()

The next cell prints an overview of the data included in the table.

In [ ]:
# DO NOT CHANGE

print(len(df), "rows in original CSV")
print(len(students), "students")
print(len(lectures), "lectures")
print(len(enrollments), "enrollments")

[Back](#sec_toc)

## <a class="anchor" id="sec_e">e) Store Tables in a Database</a>

Instead of storing data in many CSV files,
we now store it in a **database**, which allows us to manage structured data more efficiently.

In [ ]:
# DO NOT CHANGE

conn = sqlite3.connect("university.db")
students.to_sql("Student", conn, if_exists="replace", index=False)
majors.to_sql("Major", conn, if_exists="replace", index=False)
departments.to_sql("Department", conn, if_exists="replace", index=False)
faculty.to_sql("Faculty", conn, if_exists="replace", index=False)
lectures.to_sql("Lecture", conn, if_exists="replace", index=False)
enrollments.to_sql("Enrollment", conn, if_exists="replace", index=False)
conn.close()

[Back](#sec_toc)

## <a id="sec_f"> f) Compare Memory Requirements</a>

To measure the effectiveness of our enhancements, it would be interesting to compare which of these needs more memory.

The following code compared the required memory of the original CSV-table and the database. 

In [ ]:
# DO NOT CHANGE

csv_size = os.path.getsize("university_big.csv")
db_size = os.path.getsize("university.db")

print("Memory requirements:")
print(f"CSV-table: {csv_size / 1024:.2f} KB")
print(f"Database:  {db_size / 1024:.2f} KB")

To make the difference more clear, let's try the whole procedure with a larger set of students. 

In [ ]:
# DO NOT CHANGE

df_large = pd.read_csv("university_very_big.csv")

conn = sqlite3.connect("university_large.db")

students_large = df_large[[
    "student_name",
    "matriculation_number",
    "major_number"
]].drop_duplicates()
students_large.to_sql("Student", conn, if_exists="replace", index=False)

majors_large = df_large[[
    "major_name",
    "major_number",
    "degree_type"
]].drop_duplicates()
majors_large.to_sql("Major", conn, if_exists="replace", index=False)

departments_large = df_large[[
    "department_name",
    "dep_number"
]].drop_duplicates()
departments_large.to_sql("Department", conn, if_exists="replace", index=False)

faculty_large = df_large[[
    "faculty_name",
    "personnel_number",
    "role",
    "department_head",
    "dep_number"
]].drop_duplicates()
faculty_large.to_sql("Faculty", conn, if_exists="replace", index=False)

lectures_large = df_large[[
    "lecture_name",
    "lecture_number",
    "personnel_number",
    "dep_number"
]].drop_duplicates()
lectures_large.to_sql("Lecture", conn, if_exists="replace", index=False)

enrollments_large = df_large[[
    "lecture_number",
    "matriculation_number"
]].drop_duplicates()
enrollments_large.to_sql("Enrollment", conn, if_exists="replace", index=False)

conn.close()

print(len(df_large), "rows in original CSV")
print(len(students_large), "students")
print(len(lectures_large), "lectures")
print(len(enrollments_large), "enrollments")

The following code cell prints the memory requirements of the larger dataset.

In [ ]:
# DO NOT CHANGE

csv_large_size = os.path.getsize("university_very_big.csv")
db_large_size = os.path.getsize("university_large.db")

print("Memory requirements:")
print(f"CSV-table: {csv_large_size / 1024:.2f} KB")
print(f"Database:  {db_large_size / 1024:.2f} KB")

[Back](#sec_toc)